In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd

# CHANGE THIS to the folder where CSVs are
BASE_PATH = "/content/drive/MyDrive/ML_project/data/f1_2025"

def p(filename):
    return os.path.join(BASE_PATH, filename)

Mounted at /content/drive


In [2]:
#change the years
race = pd.read_csv(p("f1_race_data_2025_detailed.csv"))
laps = pd.read_csv(p("laps_vegas_2025.csv"))
pit  = pd.read_csv(p("pitstops_vegas_2025.csv"))
quali = pd.read_csv(p("qualifying_vegas_2025.csv"))
weather = pd.read_csv(p("weather_vegas_2025.csv"))

In [3]:
def time_to_seconds(t):
    """Convert 'M:SS.mmm' (or 'SS.mmm') to seconds as float."""
    if pd.isna(t):
        return np.nan
    t = str(t).strip()
    if ":" in t:
        m, s = t.split(":")
        return int(m) * 60 + float(s)
    else:
        return float(t)

# Apply to laps
laps["lapTime_sec"] = laps["lapTime"].apply(time_to_seconds)


In [4]:
laps_agg = (
    laps
    .groupby(["season", "round", "driverId"], as_index=False)
    .agg(
        laps_total=("lap", "max"),                         # last lap number
        laps_count=("lap", "count"),                       # number of laps recorded
        lapTime_mean_sec=("lapTime_sec", "mean"),
        lapTime_best_sec=("lapTime_sec", "min"),
        lapTime_std_sec=("lapTime_sec", "std"),
    )
)


In [5]:
# duration is in seconds as string (e.g., "25.201")
pit["duration_sec"] = pd.to_numeric(pit["duration"], errors="coerce")

pit_agg = (
    pit
    .groupby(["season", "round", "driverId"], as_index=False)
    .agg(
        pit_stop_count=("stop", "count"),
        pit_total_duration_sec=("duration_sec", "sum"),
        pit_avg_duration_sec=("duration_sec", "mean"),
        pit_min_duration_sec=("duration_sec", "min"),
        pit_first_lap=("lap", "min"),
        pit_last_lap=("lap", "max"),
    )
)


In [6]:
quali_small = quali[
    [
        "season", "round",
        "Driver.code", "driverId",
        "qualiPosition", "Q1_time", "Q2_time", "Q3_time"
    ]
].copy()


In [7]:
weather_renamed = weather.rename(columns={
    "airTemp_mean": "weather_airTemp_mean",
    "airTemp_min": "weather_airTemp_min",
    "airTemp_max": "weather_airTemp_max",
    "humidity_mean": "weather_humidity_mean",
    "precip_total": "weather_precip_total",
    "pressure_mean": "weather_pressure_mean",
    "wind_mean": "weather_wind_mean",
    "weather_hours": "weather_hours"
})


In [8]:
# Race has 'year', make a 'season' column
race["season"] = race["year"]

# Merge race (driver code) with quali (driver code + driverId)
race_quali = race.merge(
    quali_small,
    left_on=["season", "round", "driver"],
    right_on=["season", "round", "Driver.code"],
    how="left",
    validate="m:1"
)


In [9]:
# Merge lap aggregates
merged = race_quali.merge(
    laps_agg,
    on=["season", "round", "driverId"],
    how="left",
    validate="m:1"
)

# Merge pit stop aggregates
merged = merged.merge(
    pit_agg,
    on=["season", "round", "driverId"],
    how="left",
    validate="m:1"
)

# Merge weather (race-level, no driver dimension)
merged = merged.merge(
    weather_renamed,
    on=["season", "round"],
    how="left",
    validate="m:m"  # many drivers per race
)


In [10]:
# Start by listing groups of columns by topic
meta_cols = [
    "season", "year", "round", "race_name",
    "driver", "driverId", "constructor"
]

race_result_cols = [
    "grid_position", "finishing_position", "points",
    "num_pit_stops", "avg_lap_time", "avg_tyre_life",
    "track_temp_mid", "air_temp_mid", "humidity_mid", "rainfall_mid"
]

quali_cols = [
    "qualiPosition", "Q1_time", "Q2_time", "Q3_time"
]

lap_cols = [
    "laps_total", "laps_count",
    "lapTime_mean_sec", "lapTime_best_sec", "lapTime_std_sec"
]

pit_cols = [
    "pit_stop_count",
    "pit_total_duration_sec", "pit_avg_duration_sec", "pit_min_duration_sec",
    "pit_first_lap", "pit_last_lap"
]

weather_cols = [
    "weather_hours",
    "weather_airTemp_mean", "weather_airTemp_min", "weather_airTemp_max",
    "weather_humidity_mean", "weather_precip_total",
    "weather_pressure_mean", "weather_wind_mean"
]

# Anything we forgot:
all_cols = list(merged.columns)
used = set(meta_cols + race_result_cols + quali_cols + lap_cols + pit_cols + weather_cols)
other_cols = [c for c in all_cols if c not in used]

ordered_cols = meta_cols + race_result_cols + quali_cols + lap_cols + pit_cols + weather_cols + other_cols

merged = merged[ordered_cols]


In [11]:
output_path = p("f1_2025_vegas_master_wide.csv")
merged.to_csv(output_path, index=False)
output_path

'/content/drive/MyDrive/ML_project/data/f1_2025/f1_2025_vegas_master_wide.csv'